In [9]:
import pandas as pd


df = pd.read_csv("trondheim_vaerdata_full_1.csv")

#print(df.dtypes)

df['Tidspunkt'] = pd.to_datetime(df['Tidspunkt'])
df['weekday'] = df['Tidspunkt'].dt.strftime('%A')
df['hour'] = df['Tidspunkt'].dt.hour
 
filtered_df = df[(df['weekday'].isin(['Saturday'])) & (df['hour'].between(10,16))]

filtered_df.to_csv("filtered_data_Saturday.csv", index = False)
print(filtered_df.head(10))


              Tidspunkt  Temperatur (°C)  Vindhastighet (km/t)   Trykk  \
58  2025-01-18 10:00:00              2.3                   6.8  1020.0   
59  2025-01-18 11:00:00              2.5                   4.7  1019.0   
60  2025-01-18 12:00:00              2.7                   3.6  1019.0   
61  2025-01-18 13:00:00              3.0                   5.4  1018.0   
62  2025-01-18 14:00:00              3.4                   7.6  1017.0   
63  2025-01-18 15:00:00              2.6                   8.6  1017.0   
64  2025-01-18 16:00:00              2.3                   7.9  1017.0   
226 2025-01-25 10:00:00              4.1                  19.8   975.0   
227 2025-01-25 11:00:00              4.4                  23.4   977.0   
228 2025-01-25 12:00:00              3.9                  21.6   978.0   

     Nedbør (mm)   weekday  hour  
58          0.13  Saturday    10  
59          0.13  Saturday    11  
60          0.00  Saturday    12  
61          0.00  Saturday    13  
62        

In [21]:
#Sjekke mangler i datasettet

df = pd.read_csv("filtered_data_Saturday.csv")

missing_value = df.isnull().sum()

print(missing_value)




#HÅNDTERING AV UREGELMESSIGHETER
#værdata som kan være feilmålinger, som ekstreme temperaturer osv, plutselig hopp
df = df[(df["Temperatur (°C)"]>-50) & (df["Temperatur (°C)"]<50)]
df = df[(df["Trykk"]>900) & (df["Trykk"]<1100)]
df = df[(df["Vindhastighet (km/t)"]>45) & (df["Vindhastighet (km/t)"]<0)]

#erstatt feil med NaN
df.replace(-9999, None, inplace=True)
df["TempDiff"] = df["Temperatur (°C)"].diff() #beregn differansen mellom hver måling
df["UnormalHoppTemp"] = df["TempDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer

df["PressureDiff"] = df["Trykk"].diff() #differanse mellom hver måling
df["UnormalHoppPressure"] = df["PressureDiff"].apply(lambda x: "Ja" if abs(x) > 100 else "Nei") #Markerer plutselige endringer


df["Wind_speedDiff"] = df["Vindhastighet (km/t)"].diff()  #differanse mellom hver måling
df["UnormalHoppWind"] = df["Wind_speedDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer

df["NedborDiff"] = df["Nedbør (mm)"].diff()  #differanse mellom hver måling
df["UnormalHoppNedbor"] = df["NedborDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer






Tidspunkt               0
Temperatur (°C)         0
Vindhastighet (km/t)    0
Trykk                   0
Nedbør (mm)             0
weekday                 0
hour                    0
dtype: int64


In [4]:
#Sammenligne Pandas med Pandasql

df["TemperaturKategori"] = ["Kaldt" if temp2 < 10
                            else "mildt" if temp2 < 20 
                            else "varmt" for temp2 in df["Temperatur (°C)"]
                            ]

df["VindKategori"] = ["Vindstille" if vind2 < 2
                      else "Lett bris" if vind2 < 10
                      else "Sterk vind" if vind2 < 20
                      else "Storm" for vind2 in df["Vindhastighet (km/t)"]]

df["TrykkKategori"] = ["Høyttrykk" if trykk2 > 1013
                       else "Lavtrykk" for trykk2 in df["Trykk"]]

df["NedborKategori"] = ["Tørt" if nedbor2 == 0
                        else "Lett regn" if nedbor2 < 5
                        else "Mye regn" if nedbor2 < 20
                        else "Ekstrem vær" for nedbor2 in df["Nedbør (mm)"]]


df.head()



,Tidspunkt,Temperatur (°C),Vindhastighet (km/t),Trykk,Nedbør (mm),weekday,hour,TemperaturKategori,VindKategori,TrykkKategori,NedborKategori
0,2025-01-18 10:00:00,2.3,6.8,1020.0,0.13,Saturday,10,Kaldt,Lett bris,Høyttrykk,Lett regn
1,2025-01-18 11:00:00,2.5,4.7,1019.0,0.13,Saturday,11,Kaldt,Lett bris,Høyttrykk,Lett regn
2,2025-01-18 12:00:00,2.7,3.6,1019.0,0.00,Saturday,12,Kaldt,Lett bris,Høyttrykk,Tørt
3,2025-01-18 13:00:00,3.0,5.4,1018.0,0.00,Saturday,13,Kaldt,Lett bris,Høyttrykk,Tørt
4,2025-01-18 14:00:00,3.4,7.6,1017.0,0.00,Saturday,14,Kaldt,Lett bris,Høyttrykk,Tørt
